In [1]:
import time
import matplotlib.pyplot as plt
import numpy as np
from sage.all import *
import random

# SageMath'in vektör iç çarpımını ve normunu kullanarak projeksiyon fonksiyonu
def proj(v, u):
    a = (v.inner_product(u)) / (u.norm() ** 2) * u
    return a

# SageMath'in vektör normlarını ve iç çarpımlarını kullanarak Gram-Schmidt ortogonalizasyonu
def Gram_Schmidt(S):
    n = len(S)
    E = [S[0] / S[0].norm()]
    for k in range(1, n):
        q = S[k] - sum([proj(S[k], E[i]) for i in range(k)])
        E.append(q / q.norm())
    return E

# SageMath'in rastgele vektör oluşturma ve matris derecesi hesaplama fonksiyonlarını kullanarak bazis oluşturma
def generate_basis(W):
    new_basis = []
    while len(new_basis) < W.dimension():
        v = W.random_element()
        if v not in new_basis and v != 0:
            temp_basis = new_basis + [v]
            if Matrix(temp_basis).rank() == len(temp_basis):
                new_basis.append(v)
    return new_basis

# SageMath'in bazis oluşturma fonksiyonunu kullanarak birden fazla bazis oluşturma
def generate_multiple_bases(W, num_bases):
    bases = []
    for _ in range(num_bases):
        bases.append(generate_basis(W))
    return bases

# SageMath'in polinom halkası ve rastgele polinom oluşturma fonksiyonunu kullanarak rastgele polinom oluşturma
def get_random_func(fd):
    R.<x> = PolynomialRing(RealField(1000))
    return R.random_element(degree=fd)

# SageMath'in vektör ve reel sayı hesaplama fonksiyonlarını kullanarak katsayı hesaplama
def compute_coefficients_proj(x):
    n = len(x)
    R = RealField(1000)
    L = vector(R, n)
    for i in range(n):
        L_i = R(1)
        for j in range(n):
            if i != j:
                L_i *= (-x[j] / (x[i] - x[j]))
        L[i] = L_i
    return L

def compute_coefficients_lag(x, prime):
    n = len(x)
    L = []
    for i in range(n):
        L_i = 1
        for j in range(n):
            if i != j:
                ust = -x[j] 
                alt = (x[i] - x[j]) 
                # alt'ın tersini hesapla
                alt_inverse = pow(alt, -1, prime)
                L_i = (L_i * (ust * alt_inverse)) % prime
        L.append(L_i % prime)
    return L


In [2]:
# Group Authantication Yeni çalışma

def perform_experiment_fast_V2(num_bases,prec):
    # prec bu olunca group auth sağlanmıyor ama test için düşürdük
    R = RealField(prec)
    # Test değerleri
    d = 10  # Bu E'nin dimensionı
    E = VectorSpace(R, d)  # SageMath'in vektör uzayı oluşturma fonksiyonu
    n = 7   # Bu W'nin dimensionı
    subspace = [E.random_element() for _ in range(n)]  # SageMath'in rastgele vektör oluşturma fonksiyonu
    W = E.span(subspace)  # SageMath'in altuzay oluşturma fonksiyonu

    # Birden fazla baz oluşturma
    bases = [W.basis() for _ in range(n)]

    # Gram-Schmidt ile ortogonal bazlar oluşturma
    orthogonal_bases = [Gram_Schmidt(basis) for basis in bases]

    # Rastgele bir baz seçimi
    secret_basis = choice(orthogonal_bases)

    random_polys = get_random_func(10)

    # GM_results'in ilk değerlerini oluşturma
    GM_results = random_polys(0)

    x_i = np.random.choice(range(1, 1500), num_bases, replace=False)
    x_i = list(map(ZZ, x_i))
    
    part = 50
    segments = [x_i[i:i + part] for i in range(0, len(x_i), part)]
    g = E.random_element()  # GM yapıyor bunu SageMath'in rastgele vektör oluşturma fonksiyonu
    
    ###################################################################################
    user_time = 0
    gm_time = 0
    #auth_time = 0
    auth_time_start = time.time()
    for segment in segments:
        
        fonk_results = [random_polys(x) for x in segment]
        product_results = [[val * sb for sb in secret_basis[0]] for val in fonk_results]
        
        # auth_time_start = time.time()
        start_time_user = time.time()
        # PART 1: USER
        coefficients = compute_coefficients_proj(segment)
        coefficients_dict = {f'T_{i + 1}': coefficients[i] for i in range(len(coefficients))}
        
        # PART 2: USER
        scaled_product_results = [[coefficients[i] * value for value in product_results[i]] for i in range(len(product_results))]
        
        # PART 3: USER
        inner_product_results = [vector(sublist).inner_product(g) for sublist in scaled_product_results]
        
        end_time_user = time.time()
        user_time += (end_time_user - start_time_user) / part
        
        # PART 4: GM - VERIF
        start_time_gm = time.time() 
        sum_inner_product_results = sum(inner_product_results)
        inner_products_secret = g.inner_product(vector(secret_basis[0])) * GM_results
        
        end_time_gm = time.time() 
        gm_time += (end_time_gm - start_time_gm)
        
        #auth_time_end = time.time()
        #auth_time += (auth_time_end - auth_time_start)
        
        tolerance = 0.00001
        if abs(inner_products_secret - sum_inner_product_results) < tolerance:
            sonuc = "OK"
        else:
            sonuc = "Error"
            break
    
    #print("Geçen Süre:", total_time)
    ###################################################################################
    auth_time_end = time.time()
    auth_time = (auth_time_end - auth_time_start)
    
    return [gm_time * 1000 ,user_time * 1000 ,auth_time * 1000]
"""
def verify_bases(num_bases):
    part = 50 
    segments = num_bases // part
    total_time_elapsed = 0
    for i in range(segments):
        segment_bases = (i + 1) * part
        result = perform_experiment_fast_V2(segment_bases)
        total_time += result[0]
 
        #if result[1] != "OK":
            #print(f"{segment_bases} doğrulama hatası!")
            #break
        #print(f"{segment_bases} doğrulandı")
    if result[1] == "OK":
        print(f"{num_bases} tüm segmentler doğrulandı")
        print("Geçen Süre:", total_time_elapsed)
    return total_time_elapsed * 1000
"""
num_bases_values = num_bases_values = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
# 0. index: GM ve 1.index:User gibi düşülmelidir
times_1 = [perform_experiment_fast_V2(num_bases,5) for num_bases in num_bases_values]

for i, result in enumerate(times_1):
    user_time, gm_time, auth_time = result
    print(f"Result {i+1}:")
    print(f"User Time: {user_time}")
    print(f"GM Time: {gm_time}")
    print(f"Auth Time: {gm_time}")
    print()

Result 1:
User Time: 0.30517578125
GM Time: 0.1571035385131836
Auth Time: 0.1571035385131836

Result 2:
User Time: 0.09918212890625
GM Time: 0.14374256134033203
Auth Time: 0.14374256134033203

Result 3:
User Time: 0.09918212890625
GM Time: 0.13423919677734375
Auth Time: 0.13423919677734375

Result 4:
User Time: 0.10800361633300781
GM Time: 0.13132095336914062
Auth Time: 0.13132095336914062

Result 5:
User Time: 0.08988380432128906
GM Time: 0.13219833374023438
Auth Time: 0.13219833374023438

Result 6:
User Time: 0.09393692016601562
GM Time: 0.1327800750732422
Auth Time: 0.1327800750732422

Result 7:
User Time: 0.13017654418945312
GM Time: 0.14788150787353516
Auth Time: 0.14788150787353516

Result 8:
User Time: 0.15997886657714844
GM Time: 0.14879703521728516
Auth Time: 0.14879703521728516

Result 9:
User Time: 0.1800060272216797
GM Time: 0.14812469482421875
Auth Time: 0.14812469482421875

Result 10:
User Time: 0.13113021850585938
GM Time: 0.14196395874023438
Auth Time: 0.141963958740234

In [3]:
def check_list(inner_product):
    epsilon = 0.00001
    if len(inner_product) == 0:
        return "Not OK"  # Boş liste için
    
    first_value = inner_product[0]
    for value in inner_product:
        if abs(value - first_value) > epsilon:
            return "Not OK"
    return "OK"

#Group Authentication and Key Establishment Scheme

def sueda_enver(num_bases,prec):
    R = RealField(prec)
    
    d = 10  # Bu E'nin dimensionı
    E = VectorSpace(R, d)  # SageMath'in vektör uzayı oluşturma fonksiyonu
    n = 7   # Bu W'nin dimensionı
    subspace = [E.random_element() for _ in range(n)]  # SageMath'in rastgele vektör oluşturma fonksiyonu
    W = E.span(subspace)  # SageMath'in altuzay oluşturma fonksiyonu

    # Birden fazla baz oluşturma
    bases = [W.basis() for _ in range(n)]

    # Gram-Schmidt ile ortogonal bazlar oluşturma
    orthogonal_bases = [Gram_Schmidt(basis) for basis in bases]

    # Rastgele bir baz seçimi
    secret_basis = random.choice(orthogonal_bases)

    random_polys = get_random_func(10)

    x_i = np.random.choice(range(1, 1500), num_bases, replace=False)
    x_i = list(map(ZZ, x_i))

    r_i = [1]  # İlk eleman 1
    r_i += [random.randint(2, 1000000) for _ in range(n - 1)]
    
    v = E.random_element()
    g = E.random_element()
    
    results = [random_polys(x) for x in x_i]
    r_v = [r_i[i] * secret_basis[i] for i in range(len(r_i))]

    final_bases = [[res * sb for sb in r_v] for res in results]
    
    ###################################################################################
    start_time = time.time()
    projection = [sum([proj(v, i) for i in fb]) for fb in final_bases]
    inner_product = [proj_l.inner_product(g) for proj_l in projection]
    end_time = time.time()
    ###################################################################################
    elapsed_time = end_time - start_time
    print(num_bases,check_list(inner_product))
    print(elapsed_time)
    # 0. index: GM ve 1.index:User gibi düşülmelidir
    return [(elapsed_time * 1000),(elapsed_time * 1000)/num_bases,elapsed_time * 1000]

In [4]:
times_2 = [sueda_enver(num_bases,5) for num_bases in num_bases_values]

100 Not OK
0.030147790908813477
200 Not OK
0.05950617790222168
300 Not OK
0.08713769912719727
400 Not OK
0.11464095115661621
500 Not OK
0.14630722999572754
600 Not OK
0.2278299331665039
700 Not OK
0.20760822296142578
800 Not OK
0.2500638961791992
900 Not OK
0.251967191696167
1000 Not OK
0.27896714210510254


In [5]:
#A Flexible and Lightweight Group Authentication Scheme

def yucel_enver(uc):
    prime = 2 ** 256 - 2 ** 32 - 2 ** 9 - 2 ** 8 - 2 ** 7 - 2 ** 6 - 2 ** 4 - 1

    x_i = np.random.choice(range(1, 1500), uc, replace=False)
    x_i = list(map(ZZ, x_i))


    E = EllipticCurve(GF(prime), [0,0,0,0,7])

    P = E(55066263022277343669578718895168534326250603453777594175500187360389116729240 
      , 32670510020758816978083085130507043184471273380659243275938904335757337482424)

    order = P.order()
    R.<x> = PolynomialRing(GF(order))
    f = x^3 + 34
    

    fonk_results = [ f(x=x_val)  for x_val in x_i]
    auth_start_time = time.time()
    ###################################################################################
    user_start_time = time.time()
    coef = compute_coefficients_lag(x_i, order)
    total_result = []
    for i in range(uc):
        result = int((fonk_results[i] * coef[i])) % order
        total_result.append(result)
    GM_start_time = time.time()
    GM_results = [ P * i  for i in total_result] 
    user_end_time = time.time()  
    
    total_sum = E(0)  # Sonsuzluk noktası
    for point in GM_results:
        total_sum += point
  
    GM_end_time = time.time()
    ###################################################################################
    auth_end_time = time.time()
    auth_time = auth_end_time - auth_start_time
    GM_time = GM_end_time - GM_start_time
    user_time = user_end_time - user_start_time
    if f(0) * P == total_sum:
        print(uc,"Doğrulandı.")
    print(GM_time,user_time)
    # 0. index: GM ve 1.index:User gibi düşülmelidir
    return [GM_time * 1000 ,(user_time * 1000)/uc, auth_time * 1000 ]

In [6]:
times_3 = [yucel_enver(num_bases) for num_bases in num_bases_values]

100 Doğrulandı.
0.079193115234375 0.11301803588867188
200 Doğrulandı.
0.14354777336120605 0.2774040699005127
300 Doğrulandı.
0.2159559726715088 0.5292520523071289
400 Doğrulandı.
0.29218626022338867 0.8420560359954834
500 Doğrulandı.
0.3549368381500244 1.2409639358520508
600 Doğrulandı.
0.45200014114379883 1.783837080001831
700 Doğrulandı.
0.5441122055053711 2.3703479766845703


KeyboardInterrupt: 

In [ ]:
times_1

In [ ]:
times_2

In [ ]:
times_3

In [ ]:
# 0. index: GM ve 1.index:User gibi düşülmelidir

gm_times_1 = [x[0] for x in times_1]
user_times_1 = [x[1] for x in times_1]
auth_times_1 = [x[2] for x in times_1]

gm_times_2 = [x[0] for x in times_2]
user_times_2 = [x[1] for x in times_2]
auth_times_2 = [x[2] for x in times_2]

gm_times_3 = [x[0] for x in times_3]
user_times_3 = [x[1] for x in times_3]
auth_times_3 = [x[2] for x in times_3]

In [ ]:
# Grafik oluşturma - GM

plt.figure(figsize=(10, 6))
plt.plot(num_bases_values, gm_times_1, marker='o', label='Proposed Method')
plt.plot(num_bases_values, gm_times_2, marker='x', label='Guzey et al [19]')
plt.plot(num_bases_values, gm_times_3, marker='*',label='Aydin et al [20]')  

plt.yscale('log')
plt.xlabel('Number of User')
plt.ylabel('Time (ms)')
plt.title('GM Performance Comparison')
plt.xticks(num_bases_values)
plt.ylim(1e-2, 1e4)
plt.grid(True, which="both", ls="--")
plt.legend()
plt.show()

In [ ]:
# Grafik oluşturma - USER

plt.figure(figsize=(10, 6))
plt.plot(num_bases_values, user_times_1, marker='o', label='Proposed Method')
plt.plot(num_bases_values, user_times_2, marker='x', label='Guzey et al [19]')
plt.plot(num_bases_values, user_times_3, marker='*',label='Aydin et al [20]')  

plt.yscale('log')
plt.xlabel('Number of User')
plt.ylabel('Time (ms)')
plt.title('User Performance Comparison')
plt.xticks(num_bases_values)
plt.ylim(1e-2, 1e4)
plt.grid(True, which="both", ls="--")
plt.legend()
plt.show()

In [ ]:
# Grafik oluşturma - TOTAL_AUTH - prec=5

plt.figure(figsize=(10, 6))
plt.plot(num_bases_values, auth_times_1, marker='o', label='Proposed Method')
plt.plot(num_bases_values, auth_times_2, marker='x', label='Guzey et al [19]')
plt.plot(num_bases_values, auth_times_3, marker='*',label='Aydin et al [20]')  

plt.yscale('log')
plt.xlabel('Number of User')
plt.ylabel('Time (ms)')
plt.title('Total Performance Comparison prec=5')
plt.xticks(num_bases_values)
plt.ylim(1e-2, 1e4)
plt.grid(True, which="both", ls="--")
plt.legend()
plt.show()

In [ ]:
times_4 = [perform_experiment_fast_V2(num_bases,1000) for num_bases in num_bases_values]
times_5 = [sueda_enver(num_bases,1000) for num_bases in num_bases_values]

In [ ]:
gm_times_4 = [x[0] for x in times_4]
user_times_4 = [x[1] for x in times_4]
auth_times_4 = [x[2] for x in times_4]

gm_times_5 = [x[0] for x in times_5]
user_times_5 = [x[1] for x in times_5]
auth_times_5 = [x[2] for x in times_5]

In [ ]:
# Grafik oluşturma - TOTAL_AUTH - prec=1000

plt.figure(figsize=(10, 6))
plt.plot(num_bases_values, auth_times_4, marker='o', label='Proposed Method')
plt.plot(num_bases_values, auth_times_5, marker='x', label='Guzey et al [19]')
plt.plot(num_bases_values, auth_times_3, marker='*',label='Aydin et al [20]')  

plt.yscale('log')
plt.xlabel('Number of User')
plt.ylabel('Time (ms)')
plt.title('Total Performance Comparison prec=1000')
plt.xticks(num_bases_values)
plt.ylim(1e1, 1e4)
plt.grid(True, which="both", ls="--")
plt.legend()
plt.show()